# **Challenge 3: Retrieve search output and interact with Azure OpenAI**

### Azure AI Search Method

Azure AI Search service enhances search capabilities by offering various methods such as pure vector search, hybrid search, and semantic hybrid search.

- **Pure Vector Search:** Utilizes vector representations of documents to perform searches based on semantic similarity. This method is ideal for applications requiring high precision in finding relevant passages or documents through semantic matching.
    - Use case: Matching image content with text content, cross-lingual searches, or finding similar documents.
- **Hybrid Search**: Combines traditional keyword search with vector search, applying a fusion step to select the best results from each technique. It’s particularly effective for general search use cases where both relevance and keyword matching are important.
    - Use case: Retrieval-augmented generation applications, where both the retrieval of relevant documents and the ranking based on semantic relevance are key.
- **Semantic Hybrid Search**: Integrates semantic ranking with hybrid search to further refine the relevance of search results. This approach is most beneficial for Generative AI scenarios, where precise retrieval of information is crucial for generating accurate responses.
    - Use case: Enhancing chat-style and copilot applications by ensuring the top search results are the most semantically relevant to the query.

## Let start the challenge

In [1]:
# Import required libraries  
import os
import textwrap
from openai import AzureOpenAI, DefaultHttpxClient
from dotenv import load_dotenv, find_dotenv
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector

In [2]:
# Configure environment variables  
load_dotenv(find_dotenv('credential.env'), override=True)

# Azure AI Search
service_endpoint = os.environ['AZURE_AI_SEARCH_ENDPOINT']
key = os.environ['AZURE_AI_SEARCH_KEY']
index_name = os.environ['AZURE_AI_SEARCH_INDEX_NAME']
credential = AzureKeyCredential(key)

#Azure OpenAI
client = AzureOpenAI(
  api_key = os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  azure_endpoint = os.environ['AZURE_OPENAI_API_ENDPOINT'],
  api_version = os.environ['AZURE_OPENAI_API_VERSION'],
  http_client=DefaultHttpxClient(verify=False)
)
embedding_model = os.environ['EMBEDDING_MODEL_NAME']
gpt35turbo_model = os.environ['GPT35TURBO_MODEL_NAME']
gpt4turbo_model = os.environ['GPT4TURBO_MODEL_NAME']
gpt4o_model = os.environ['GPT4O_MODEL_NAME']

In [3]:
# Declare useful method
def check_and_create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"The folder '{folder_name}' has been created.")
    else:
        print(f"The folder '{folder_name}' already exists.")

def print_error_message(message, prefix_message='Error: '):
    print(f"\033[1;31m{prefix_message}\033[0m{message}")

def print_warning_message(message, prefix_message='Warning: '):
    print(f"\033[1;33m{prefix_message}\033[0m{message}")
    
def print_success_message(message, prefix_message='Success: '):
    print(f"\033[1;32m{prefix_message}\033[0m{message}")

In [4]:
def generate_embeddings(text):
    response = client.embeddings.create(input=text, model=embedding_model)
    return response.data[0].embedding


def get_chat_completion(messages, model):
    print(f'Generating answer using {model} LLM model')
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

def format_retrieved_documents(data_list):
    retrieved_documents = ""
    for item in data_list:
        # Extract the 'content' and 'sourcepage' values
        content = item.get('content', '')
        sourcepage = item.get('sourcepage', '')
        retrieved_documents += "sourcepage: " + sourcepage + ", content: " + content + "\n"
    return retrieved_documents

## Perform a vector similarity search

In [ ]:
# Pure Vector Search
query = "What is GenAI?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)

# Embedding "query" before searching
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

# Only search using vector content to query agaist "contentVector" field
results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["content", "sourcepage"],
)

result_list = []
for result in results:  
    print_warning_message(f"{result['sourcepage']}", "Source Page: ")
    print_warning_message(f"{result['@search.score']}", "Search Score: ")
    print_warning_message(f"{textwrap.shorten(result['content'], width=100, placeholder="...")}\n", "Content: ")
    result_list.append(result)

print_success_message("", "-----AI Search JSON Output [Pure Vector Search]-----")
result_list

In [ ]:
# Pure Vector Search multi-lingual 
query = "GenAI คืออะไร"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  

vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["content", "sourcepage"],
)  

result_list = []
for result in results:  
    print_warning_message(f"{result['sourcepage']}", "Source Page: ")
    print_warning_message(f"{result['@search.score']}", "Search Score: ")
    print_warning_message(f"{textwrap.shorten(result['content'], width=100, placeholder="...")}\n", "Content: ")
    result_list.append(result)
    

print_success_message("", "-----AI Search JSON Output [Pure Vector Search]-----")
result_list

## Perform a Hybrid Search

In [ ]:
# Hybrid Search
query = "What is GenAI?"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  

# Embedding "query" before searching
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

# Search using as it "query" as a semantic search together with vector query
results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["content", "sourcepage"],
    top=3
)  

result_list = []
for result in results:  
    print_warning_message(f"{result['sourcepage']}", "Source Page: ")
    print_warning_message(f"{result['@search.score']}", "Search Score: ")
    print_warning_message(f"{textwrap.shorten(result['content'], width=100, placeholder="...")}\n", "Content: ")
    result_list.append(result)

print_success_message("", "-----AI Search JSON Output [Hybrid Search]-----")
result_list

## Perform a Semantic Hybrid Search

In [ ]:
# Semantic Hybrid Search
query = "what is genai?"

search_client = SearchClient(
    service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    select=["content", "sourcepage"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

result_list = []
# Retrieve top k relevance chunked document from Azure AI Search
for result in results:
    print_warning_message(f"{result['sourcepage']}", "Source Page: ")
    print_warning_message(f"{result['@search.score']}", "Search Score: ")
    print_warning_message(f"{textwrap.shorten(result['content'], width=100, placeholder="...")}", "Content: ")
    print_warning_message(f"{result['@search.reranker_score']}", "Reranker Score: ")
    result_list.append(result)

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print_warning_message(f"{caption.highlights}\n", "Caption: ")
        else:
            print_warning_message(f"{caption.text}\n", "Caption: ")

# Generate semantic answers based on top k relevance chunked documents
semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print_warning_message(f"{answer.highlights}", "Semantic Answer: ")
    else:
        print_warning_message(f"{answer.text}", "Semantic Answer: ")
    print_warning_message(f"{answer.score}\n", "Semantic Answer Score: ")

print_success_message("", "-----AI Search JSON Output [Semantic Hybrid Search with reranking]-----")
result_list

## Define system prompt for large language model

In the context of Azure OpenAI, the roles of the system, user, and assistant are crucial for the effective functioning of the AI models, particularly in chat-based interactions. Here’s a breakdown of each role and the importance of the system message:

**System:** This is a predefined message that sets the stage for the AI’s behavior. It primes the model with context, instructions, or other relevant information for the specific use case. The system message can describe the assistant’s personality, outline what the model should and shouldn’t answer, and define the format of the model’s responses. It’s a critical component because it helps ensure that the AI operates within the desired parameters and provides responses that are aligned with the user’s expectations.

**User:** The user interacts with the AI model by providing input or asking questions. The user’s messages are the prompts that the AI responds to. The quality and clarity of the user’s input can significantly influence the accuracy and relevance of the AI’s responses.

**Assistant:** The assistant is the AI itself, which processes the user’s input and the context provided by the system message to generate helpful, accurate, and contextually appropriate responses. The assistant’s role is to assist users by providing information, answering questions, and engaging in conversation within the guidelines set by the system message.

The importance of the system message lies in its ability to guide the AI system’s behavior and improve its performance. It defines the AI’s capabilities and limitations, the expected output format, and additional safety and behavioral guardrails. By crafting an effective system message, developers can increase the accuracy and grounding of the responses generated by the AI, making it a more reliable and efficient tool for users.

In [ ]:
system_prompt = f"""
## On your role
- You're an AI assistance to help answer questions based on retrieved documents.
## Retrieved documents
""" + format_retrieved_documents(result_list)

print(system_prompt)

## Call Azure OpenAI API to generate the answer based retrieved documents

In [ ]:
# Define messages format to interact with LLM model
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
]

# GPT Model Option: gpt35turbo_model, gpt4turbo_model and gpt4o_model
response = get_chat_completion(messages, gpt4o_model)
print("assistant: "+response)